# Generators

- Writing a class-based iterator requires `__iter__()` and `__next__()`, plus manual state management and `StopIteration` handling.  
- Generator functions let you express the same logic in plain Python functions, using `yield` to produce values one at a time.  
- Any function with `yield` becomes a generator: calling it returns a generator object (an iterator) without running its body immediately.  

In [1]:
def count_upt_to(limit):
    """Generates numbers from 1 to limit.
    
    Args:
        limit (int): The maximum number to count up to.

    Returns:
        generator: A generator that yields numbers from 1 to limit.    
    
    """

    print("Generator function started...")
    n = 1

    while n <= limit:
        print(f"Yielding {n}...")
        yield n
        print(f"Resuming after yielding{n}...")
        n += 1

    print("Generator function is done.")    

count_gen = count_upt_to(3)
print(f"Returned object: {count_gen} of type {type(count_gen)}")

""" current_number = next(count_gen)
print(current_number) """

for count in count_gen:
    print(f"Count: {count}")

Returned object: <generator object count_upt_to at 0x0000020BC7413D80> of type <class 'generator'>
Generator function started...
Yielding 1...
Count: 1
Resuming after yielding1...
Yielding 2...
Count: 2
Resuming after yielding2...
Yielding 3...
Count: 3
Resuming after yielding3...
Generator function is done.


## Generator Functions & the `yield` Keyword

- A function becomes a generator by including `yield`; no other boilerplate is needed.  
- Calling a generator function returns an object that implements `__iter__()` and `__next__()`.  
- The code inside runs only when iteration begins (e.g., in a `for` loop or via `next()`).  

In [3]:
def filter_evens(data):
    """Yields only the even items from the input sequence.

    Args:
        data (iterable): An iterable of integers or float.

    Returns:
        generator: A generator that yields only the even numbers from the input sequence.
    """
    for item in data:
        if item % 2 == 0:
            print(f"Yielding even item: {item}")
            yield item
    print("No more items to process.")


evens = filter_evens(range(10))

print(f"Generated object: {evens}")            

for even in evens:
    print(f"Even number: {even}")

Generated object: <generator object filter_evens at 0x0000020BC758C580>
Yielding even item: 0
Even number: 0
Yielding even item: 2
Even number: 2
Yielding even item: 4
Even number: 4
Yielding even item: 6
Even number: 6
Yielding even item: 8
Even number: 8
No more items to process.


## How `yield` Works: Pause and Resume

- On each `next()` (or loop iteration), execution runs until it hits `yield`, returns the value, then pauses with all local state intact.  
- The next `next()` call resumes immediately after the `yield`, preserving variables and the instruction pointer.  
- When the function ends (no more `yield`), a `StopIteration` is raised automatically.  

In [4]:
def demo_three_yields():
    print("Starting demo_three_yields")
    yield "First yield"
    print("Between first and second yield")
    yield "Second yield"
    print("Between second and third yield")
    yield "Third yield"
    print("End of demo_three_yields")

demo_gen= demo_three_yields()
print(next(demo_gen))
print(next(demo_gen))
print(next(demo_gen))    

Starting demo_three_yields
First yield
Between first and second yield
Second yield
Between second and third yield
Third yield


## Generator State

- Generators keep their local variables alive between yields, making explicit state objects unnecessary.  
- This persistent state allows infinite or long-running sequences without full data storage.  

In [5]:
count_gen = count_upt_to(3)

print("First call to  next outsideof the loop.")
print(next(count_gen))

print("Second call to next outside of the loop - now the value  yielded will be 2.")
print(next(count_gen))

print("Now continuing with the for loop.")
for count in count_gen:
    print(f"Count: {count}")

First call to  next outsideof the loop.
Generator function started...
Yielding 1...
1
Second call to next outside of the loop - now the value  yielded will be 2.
Resuming after yielding1...
Yielding 2...
2
Now continuing with the for loop.
Resuming after yielding2...
Yielding 3...
Count: 3
Resuming after yielding3...
Generator function is done.


In [6]:
count_gen = count_upt_to(5)

# Since generattors have state, using the same generator in nested loops will not work as expected.
# The inner for loop will complete its iteration for each value of the outer loop. and the outer for loop will have single iteration.
for num in count_gen:
    for num2 in count_gen:
        print(f" - {num}:{num2}")

# Since generators have state, using two different generator instances in nested loops will work as expected.
# Each generator will maintain its own state and will iterate independently.
for num in count_upt_to(5):
    for num2 in count_upt_to(5):
        print(f" - {num}:{num2}")        

Generator function started...
Yielding 1...
Resuming after yielding1...
Yielding 2...
 - 1:2
Resuming after yielding2...
Yielding 3...
 - 1:3
Resuming after yielding3...
Yielding 4...
 - 1:4
Resuming after yielding4...
Yielding 5...
 - 1:5
Resuming after yielding5...
Generator function is done.
Generator function started...
Yielding 1...
Generator function started...
Yielding 1...
 - 1:1
Resuming after yielding1...
Yielding 2...
 - 1:2
Resuming after yielding2...
Yielding 3...
 - 1:3
Resuming after yielding3...
Yielding 4...
 - 1:4
Resuming after yielding4...
Yielding 5...
 - 1:5
Resuming after yielding5...
Generator function is done.
Resuming after yielding1...
Yielding 2...
Generator function started...
Yielding 1...
 - 2:1
Resuming after yielding1...
Yielding 2...
 - 2:2
Resuming after yielding2...
Yielding 3...
 - 2:3
Resuming after yielding3...
Yielding 4...
 - 2:4
Resuming after yielding4...
Yielding 5...
 - 2:5
Resuming after yielding5...
Generator function is done.
Resuming aft

## Exhaustion

- Once a generator’s code path completes (falls off the end or hits `return`), further `next()` calls immediately raise `StopIteration`.  
- A `for` loop over an exhausted generator does nothing on subsequent passes—you must call the function again for a fresh iterator.  

In [7]:
count_gen = count_upt_to(3)
print(next(count_gen))
print(next(count_gen))

try:
    print(next(count_gen))
    print(next(count_gen))  # This will raise StopIteration
except StopIteration:
    print("Reached the end of the generator.")

for number in count_gen:
    print(f"Number: {number}")  # This will not print anything as the generator is exhausted        

Generator function started...
Yielding 1...
1
Resuming after yielding1...
Yielding 2...
2
Resuming after yielding2...
Yielding 3...
3
Resuming after yielding3...
Generator function is done.
Reached the end of the generator.
